In [ ]:
from collections import deque

def get_peaksDict(w, positions):
    '''
    A helper function that computes the levels of given positions in a Dyck word w.
    w: Dyckword. The input Dyck word.
    positions: set. The positions to compute levels for.
    Returns: list. The levels of the given positions.
    '''
    level = 0
    peaksDict = {}
    for i in range(len(w)):
        if w[i] == 1:
            level += 1
        else:
            level -= 1
        if i < len(w) and w[i] == 1 and w[i+1] == 0:
            peaksDict.setdefault(level, deque([])).appendleft(i)
    return peaksDict

def scaffolding(w):
    '''
    Returns the output of Zeta map via a "scaffolding" algorithm that only reads information from the right
    once levels are computed. We are obeying the south-west level convention.

    w: list. The input Dyck word.
    '''
    n = len(w)  # semi-length of w

    # 1. Extract the right steps's positions:
    Rs_pos = set(i for i in range(n) if w[i] == 0)

    # 2. Construct the levels for the rights:
    peaksDict = get_peaksDict(w,Rs_pos)   # levels is a helper function that returns the levels of given positions

    # 3. The scaffolding algorithm:
    out = []    # to return result
    queue = []  # a mostly sorted queue that stores tuples (level, position)
                # to be visited, where insertions might also be performed.
    cur_level = max(peaksDict)  # to keep track of which level we are iterating
    alive_agents = deque([])   # to keep track of where we are on the scaffolding. Format: [pos, if_right]

    while len(out) < n:

        peaks = peaksDict.get(cur_level, [])
        if peaks:    # if there are peaks of this level
            queue += peaks    # append the ups before them to queue

        if alive_agents:
            queue += alive_agents  # adding valid agents to queue

        queue.sort(reverse=True)        # sort the same level step by their positions
                                        # this should be fast as it is merging two sorted lists

        out += [w[pos] for pos in queue]    # append this to output
        queue = []              # clear queue

        # now let our agents on the scaffoldings observe and move
        # move existing ones first
        if alive_agents:
            for i in range(len(alive_agents)):  # iterate backwards to allow popping
                agent_pos = alive_agents.popleft()
                if agent_pos in Rs_pos:  # agent is moving right
                    agent_pos = agent_pos + 1
                    if agent_pos in Rs_pos and agent_pos < 2*n: # so it's valid
                        alive_agents.append(agent_pos)  # move agent
                else:   # moving left
                    agent_pos = agent_pos - 1
                    if agent_pos not in Rs_pos and agent_pos >= 0:
                        alive_agents.append(agent_pos)

        # then we add two agents to the right and left of peak
        for peaks_pos in peaks:
            agent_pos = peaks_pos + 1
            if agent_pos in Rs_pos and agent_pos < 2*n:
                alive_agents.append(agent_pos)
            agent_pos = peaks_pos - 1
            if agent_pos not in Rs_pos and agent_pos >= 0:
                alive_agents.append(agent_pos)
        # update current level
        cur_level -= 1
    
    return out

In [ ]:
from collections import deque, defaultdict
from typing import List, Dict, Sequence

def _peaks_by_level_desc(w: List[int]) -> Dict[int, List[int]]:
    n2 = len(w)
    level = 0
    peaks = defaultdict(list)
    for i in range(n2 - 1):  # guard i+1
        level += 1 if w[i] == 1 else -1
        if w[i] == 1 and w[i + 1] == 0:
            peaks[level].append(i)
    for L in peaks:
        peaks[L].reverse()  # make positions descending
    return peaks

def _merge_desc(into: List[int], a: Sequence[int], b: Sequence[int]) -> List[int]:
    """
    Merge two descending sequences a and b into 'into' (descending), without slicing.
    Works for list or deque.
    """
    i = j = 0
    la, lb = len(a), len(b)
    # step through both
    while i < la and j < lb:
        va, vb = a[i], b[j]
        if va >= vb:
            into.append(va); i += 1
        else:
            into.append(vb); j += 1
    # drain remainder (no slicing!)
    while i < la:
        into.append(a[i]); i += 1
    while j < lb:
        into.append(b[j]); j += 1
    return into

def get_peaksDict(w: List[int], positions=None):
    peaks = _peaks_by_level_desc(w)
    return {L: deque(vs) for L, vs in peaks.items()}

def scaffolding_v1(w: List[int]) -> List[int]:
    n2 = len(w)
    Rs = [step == 0 for step in w]  # bool array for O(1) membership

    peaksDict = _peaks_by_level_desc(w)  # lists (descending)
    cur_level = max(peaksDict) if peaksDict else 0

    out: List[int] = []
    alive_agents: deque[int] = deque()     # keep descending order
    queue: List[int] = []

    while len(out) < n2:
        peaks = peaksDict.get(cur_level, [])

        queue.clear()
        if peaks and alive_agents:
            _merge_desc(queue, peaks, alive_agents)
        elif peaks:
            # already descending
            queue.extend(peaks)
        elif alive_agents:
            # already descending
            queue.extend(alive_agents)

        # write output for this round
        out.extend(w[pos] for pos in queue)

        # move existing agents
        if alive_agents:
            moved = deque()
            for _ in range(len(alive_agents)):
                pos = alive_agents.popleft()
                if Rs[pos]:                # moving right
                    np = pos + 1
                    if np < n2 and Rs[np]:
                        moved.append(np)
                else:                      # moving left
                    np = pos - 1
                    if np >= 0 and not Rs[np]:
                        moved.append(np)
            alive_agents = moved

        # spawn around peaks
        if peaks:
            for p in peaks:
                pr = p + 1
                if pr < n2 and Rs[pr]:
                    alive_agents.append(pr)
                pl = p - 1
                if pl >= 0 and not Rs[pl]:
                    alive_agents.append(pl)
            # one sort per level step (could be optimized further if needed)
            alive_agents = deque(sorted(alive_agents, reverse=True))

        cur_level -= 1

    return out

In [25]:
from sage.all import DyckWord, DyckWords

In [108]:
vecs = DyckWords(8)
for v in vecs:
    w = list(v)
    out1 = scaffolding(w)
    out2 = list(v.area_dinv_to_bounce_area_map())
    if out1 != out2:
        print("Mismatch found!")
        print("Input:", w)
        break

In [81]:
# time test
import time

n = 11
vecs = DyckWords(n)

# # randomly sample 1000 words
# import random
# vecs = random.sample(list(vecs), 1000)
# vecs = [DyckWord(list(v)) for v in vecs]

vecs_w = [list(v) for v in vecs]

start = time.time()
for v in vecs_w:
    out1 = scaffolding(v)
end = time.time()
print("Scaffolding time:", end - start)

start = time.time()
for v in vecs:
    out2 = v.area_dinv_to_bounce_area_map()
end = time.time()
print("Built-in time:", end - start)

start = time.time()
for v in vecs_w:
    out3 = scaffolding_v1(v)
end = time.time()
print("Scaffolding_v1 time:", end - start)


Scaffolding time: 2.148768901824951
Built-in time: 2.713838577270508
Scaffolding_v1 time: 1.9313626289367676


In [77]:
# get the area sequence of a Dyck word
def area_sequence(w):
    n = len(w)
    area_seq = []
    level = 0
    for i in range(n):
        if w[i] == 1:
            area_seq.append(level)
            level += 1
        else:
            level -= 1
    return area_seq

In [83]:
start = time.time()

for v in vecs:
    out1 = area_sequence(v)
end = time.time()
print("Area sequence time:", end - start)

start = time.time()
for v in vecs:
    out2 = v.to_area_sequence()
end = time.time()
print("Built-in time:", end - start)

Area sequence time: 1.8688297271728516
Built-in time: 1.4206693172454834


In [98]:
# sweep map
def dyck_ranks(w: Sequence[int]) -> List[int]:
    """
    Rank of each step = y - x at the step's start (SW endpoint).
    Assumes w is a Dyck path of length 2n from (0,0) to (n,n) staying above y=x.
    """
    ranks = []
    rank = 0
    for step in w:
        if step == 1:  # N
            rank += 1
        else:          # E
            rank -= 1
        ranks.append(rank)
    return ranks

def sweep_map(w: Sequence[int]) -> List[int]:
    """
    Sweep map on Dyck paths (Loehr–Warrington / Haglund style):
    - Compute rank r_i = y_i - x_i at the start of step i.
    - Sort steps by increasing rank; break ties by scanning **right-to-left** (i.e., larger i first).
    - Output the step multiset in that order.

    Returns a Dyck word of the same length.
    """
    n2 = len(w)
    r = dyck_ranks(w)

    # Indices 0..2n-1, tie-broken right-to-left within each rank: use (-i)
    order = sorted(range(n2), key=lambda i: (r[i], i), reverse=True)

    # Emit steps in that order (this is guaranteed to be Dyck)
    out = [w[i] for i in order]
    return out

In [99]:
vecs = DyckWords(10)
for v in vecs:
    w = list(v)
    out1 = sweep_map(w)
    out2 = list(v.area_dinv_to_bounce_area_map())
    if out1 != out2:
        print("Mismatch found!")
        print("Input:", w)
        print("Output sweep_map:", out1)
        print("Output built-in:", out2)
        break

In [109]:
vecs = DyckWords(11)
vecs_w = [list(v) for v in vecs]

start = time.time()
for v in vecs_w:
    out1 = scaffolding_v1(v)
end = time.time()
print("Scaffolding time:", end - start)

start = time.time()
for v in vecs:
    out2 = v.area_dinv_to_bounce_area_map()
end = time.time()
print("Built-in time:", end - start)

start = time.time()
for v in vecs:
    out2 = sweep_map(list(v))
end = time.time()
print("Sweep map time:", end - start)

Scaffolding time: 1.924407720565796
Built-in time: 2.6868908405303955
Sweep map time: 2.160309076309204


# Animation of Scaffolding Algorithm

In [12]:
import math
import plotly.graph_objects as go
from collections import deque
from typing import Sequence, List, Tuple

# ===============================
# Helper: compute peaks
# ===============================
def get_peaksDict(w: Sequence[int], positions) -> dict:
    level = 0
    peaksDict = {}
    for i in range(len(w) - 1):
        if w[i] == 1:
            level += 1
        else:
            level -= 1
        if w[i] == 1 and w[i + 1] == 0:
            peaksDict.setdefault(level, deque()).appendleft(i)
    return peaksDict

# ===============================
# Compute path coordinates
# ===============================
def dyck_coordinates(w: Sequence[int]) -> List[Tuple[int, int]]:
    """Convert Dyck word (1=N, 0=E) into list of lattice coordinates."""
    x = y = 0
    coords = [(x, y)]
    for step in w:
        if step == 1:
            y += 1
        else:
            x += 1
        coords.append((x, y))
    return coords

# ===============================
# Scaffolding + recording
# ===============================
def scaffolding_with_history(w: Sequence[int]) -> Tuple[List[List[int]], List[Tuple[int, int]]]:
    """
    Run the scaffolding algorithm and record alive_agents at each step.
    Returns:
      agents: list of lists of alive agent positions
      coords: path lattice coordinates
    """
    n = len(w)

    Rs_pos = {i for i, step in enumerate(w) if step == 0}
    peaksDict = get_peaksDict(w, Rs_pos)

    out: List[int] = []
    queue: List[int] = []
    cur_level = max(peaksDict) if peaksDict else 0
    alive_agents: deque[int] = deque()

    history = {'agents': [], 'dones': []}
    
    while len(out) < n:
        peaks = peaksDict.get(cur_level, [])
        if peaks:
            queue += list(peaks)

        peaks_his = []
        for p in peaks:
            peaks_his.append(p)
            history['agents'].append(list(alive_agents) + peaks_his)
            history['dones'].append(history['dones'][-1] if history['dones'] else [])

        if alive_agents:
            queue += list(alive_agents)

        queue.sort(reverse=True)

        for pos in queue:
            out.append(w[pos])
            history['dones'].append(history['dones'][-1] + [pos] if history['dones'] else [pos])
            history['agents'].append(history['agents'][-1])  # repeat last agents
        # out.extend(w[pos] for pos in queue)
        queue.clear()

        if alive_agents:
            for _ in range(len(alive_agents)):
                agent_pos = alive_agents.popleft()
                if agent_pos in Rs_pos:
                    next_pos = agent_pos + 1
                    if next_pos in Rs_pos and next_pos < 2 * n:
                        alive_agents.append(next_pos)
                        history['agents'].append(list(alive_agents))
                        history['dones'].append(history['dones'][-1] if history['dones'] else [])
                else:
                    next_pos = agent_pos - 1
                    if next_pos not in Rs_pos and next_pos >= 0:
                        alive_agents.append(next_pos)
                        history['agents'].append(list(alive_agents))
                        history['dones'].append(history['dones'][-1] if history['dones'] else [])

        for peak_pos in peaks:
            right_pos = peak_pos + 1
            if right_pos in Rs_pos and right_pos < 2 * n:
                alive_agents.append(right_pos)
                history['agents'].append(list(alive_agents))
                history['dones'].append(history['dones'][-1] if history['dones'] else [])
            left_pos = peak_pos - 1
            if left_pos not in Rs_pos and left_pos >= 0:
                alive_agents.append(left_pos)
                history['agents'].append(list(alive_agents))
                history['dones'].append(history['dones'][-1] if history['dones'] else [])
        cur_level -= 1

    return history, dyck_coordinates(w)

# ===============================
# Plotly slider visualization
# ===============================
def animate_scaffolding_plotly(w: Sequence[int], initial_frame: int = 1) -> go.Figure:
    history, coords = scaffolding_with_history(w)

    x_path, y_path = zip(*coords)

    x_out_path, y_out_path = [], []
    out_coordinates = []
    out_now = []
    for frame in range(len(history['dones'])):
        out_now.append([w[i] for i in history['dones'][frame]])
        out_coordinates.append(dyck_coordinates(out_now[-1]) if out_now[-1] else [])
    x_out_path = [list(zip(*coords_frame))[0] if coords_frame else [] for coords_frame in out_coordinates]
    y_out_path = [list(zip(*coords_frame))[1] if coords_frame else [] for coords_frame in out_coordinates]

    diag = list(range(len(coords)))

    mids: List[Tuple[float, float]] = []
    for (x1, y1), (x2, y2) in zip(coords[:-1], coords[1:]):
        mids.append(((x1 + x2) / 2.0, (y1 + y2) / 2.0 + 0.2))

    agent_positions: List[Tuple[List[float], List[float], List[str]]] = []
    dones = []
    for alive in history['agents']:
        xs: List[float] = []
        ys: List[float] = []
        labels: List[str] = []
        for pos in alive:
            if 0 <= pos < len(mids):
                x_mid, y_mid = mids[pos]
                xs.append(x_mid)
                ys.append(y_mid)

                if pos <= len(w) - 2:
                    if w[pos] == 1 and w[pos + 1] == 0:
                        labels.append("🗻")
                    elif w[pos] == 1 and w[pos + 1] == 1:
                        labels.append("🥷")
                    else:
                        labels.append("🧍")
                else:
                    labels.append("🧍")
        agent_positions.append((xs, ys, labels))
    for done in history['dones']:
        xs: List[float] = []
        ys: List[float] = []
        labels: List[str] = []
        for pos in done:
            if 0 <= pos < len(mids):
                x_mid, y_mid = mids[pos]
                xs.append(x_mid)
                ys.append(y_mid)
                labels.append("✅")
        dones.append((xs, ys, labels))

    frame_count = len(dones)
    initial_idx = max(0, min(initial_frame - 1, frame_count - 1))
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_path, y=y_path, mode="lines", name="Input", line=dict(width=3)))
    fig.add_trace(go.Scatter(x=diag, y=diag, mode="lines", name="Diagonal", line=dict(color="gray", dash="dash")))
    fig.add_trace(go.Scatter(x=x_out_path[initial_idx], y=y_out_path[initial_idx], mode="lines", name="Output", line=dict(width=3, color="orange")))

    init_xs, init_ys, init_labels = agent_positions[initial_idx]
    fig.add_trace(go.Scatter(
        x=init_xs,
        y=init_ys,
        mode="text",
        text=init_labels,
        textfont=dict(size=24),
        name="Agents",
        hoverinfo="skip"
    ))

    init_xs, init_ys, init_labels = dones[initial_idx]
    fig.add_trace(go.Scatter(
        x=init_xs,
        y=init_ys,
        mode="text",
        text=init_labels,
        textfont=dict(size=24),
        name="Done",
        hoverinfo="skip"
    ))

    slider_steps = []
    for idx in range(len(agent_positions)):
        title = f"Update {idx + 1}/{frame_count}"
        step = dict(
            method="update",
            args=[
                {'x': [list(x_path), diag, x_out_path[idx], agent_positions[idx][0], dones[idx][0]], 
                 'y': [list(y_path), diag, y_out_path[idx], agent_positions[idx][1], dones[idx][1]], 
                 'text': [None, None, None, agent_positions[idx][2], dones[idx][2]]},
                {'title': title}
            ],
            # args=[
            #     {'x': [list(x_path), diag, x_out_path[idx], agent_positions[idx][0]], 
            #      'y': [list(y_path), diag, y_out_path[idx], agent_positions[idx][1]], 
            #      'text': [None, None, None, agent_positions[idx][2]]},
            #     {'title': title}
            # ],
            label=str(idx + 1)
        )
        slider_steps.append(step)

    max_coord = min(6, math.ceil(max(max(x_path), max(y_path), max(diag))))
    x_range = [-0.5, 6]
    y_range = [-0.5, 6]

    grid_shapes = []
    for xi in range(7):
        grid_shapes.append({
            'type': 'line',
            'xref': 'x',
            'yref': 'y',
            'x0': xi,
            'x1': xi,
            'y0': y_range[0],
            'y1': y_range[1],
            'line': {'color': 'rgba(200,200,200,0.4)', 'width': 1}
        })
    for yi in range(7):
        grid_shapes.append({
            'type': 'line',
            'xref': 'x',
            'yref': 'y',
            'x0': x_range[0],
            'x1': x_range[1],
            'y0': yi,
            'y1': yi,
            'line': {'color': 'rgba(200,200,200,0.4)', 'width': 1}
        })

    fig.update_layout(
        title=f"Update {initial_idx + 1}/{frame_count} — {len(init_labels)} agents",
        xaxis=dict(title="x", range=x_range, scaleanchor="y", scaleratio=1),
        yaxis=dict(title="y", range=y_range),
        sliders=[dict(active=initial_idx, currentvalue={'prefix': 'Frame: '}, pad={'t': 50}, steps=slider_steps)],
        margin=dict(l=40, r=40, t=80, b=40),
        width=650,
        height=650,
        shapes=grid_shapes
    )
    return fig

# ===============================
# Example
# ===============================
w = [1, 0, 1, 1, 0, 1, 0, 0, 1, 0]
# w = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
fig = animate_scaffolding_plotly(w)
fig


In [13]:
# save fig
fig.write_html("scaffolding_animation_output.html")